<a href="https://colab.research.google.com/github/aethelind/notebooks-misc/blob/main/impossible_synthetic_aaai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Decision-Focused Learning Simple Synthetic Problem**

# Get required files

In [1]:
# Clear out directory
!rm -rf *
# Download data_decisions_benchmarks.zip and unzip diverse_recommendation_data.pickle
!curl https://bryanwilder.github.io/files/data_decisions_benchmarks.zip | jar xv benchmarks_release/diverse_recommendation_data.pickle
# Move diverse_recommendation_data.pickle to current directory
!mv benchmarks_release/diverse_recommendation_data.pickle .
# Remove empty directory
!rm -rf benchmarks_release
# Download hetrec2011-movielens-2k-v2.zip and unzip movie_actors.dat and user_ratedmovies.dat
!curl https://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-2k-v2.zip | jar xv movie_actors.dat user_ratedmovies.dat

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 83.0M  100 83.0M    0     0  8956k      0  0:00:09  0:00:09 --:--:-- 15.4M
 inflated: benchmarks_release/diverse_recommendation_data.pickle
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0 17.9M    0 49152    0     0  64000      0  0:04:54 --:--:--  0:04:54 63916 inflated: movie_actors.dat
 35 17.9M   35 6512k    0     0  3670k      0  0:00:05  0:00:01  0:00:04 3668k inflated: user_ratedmovies.dat
100 17.9M  100 17.9M    0     0  7440k      0  0:00:02  0:00:02 --:--:-- 7440k


# Libraries

In [2]:
## submodular.py
import torch

class ContinuousOptimizer(torch.autograd.Function):
    """
    pytorch module for differentiable submodular maximization. The forward pass 
    computes the optimal x for given parameters. The backward pass differentiates 
    that optimal x wrt the parameters.
    """

    @staticmethod
    def forward(ctx, params, optimize_func, get_dgradf_dparams, get_hessian=None, max_x=1., verbose=False):
        """
        Computes the optimal x using the supplied optimizer. 
        """
        ctx.optimize_func = optimize_func
        ctx.get_dgradf_dparams = get_dgradf_dparams
        ctx.verbose = verbose
        ctx.get_hessian = get_hessian
        ctx.all_xs = []
        ctx.max_x = max_x

        import numpy as np
        with torch.enable_grad():
            x = ctx.optimize_func(params, verbose=ctx.verbose)
        ctx.x = x.data
        ctx.all_xs.append(ctx.x.detach().numpy())
        ctx.params = params
        ctx.xgrad = x.grad.data
        return x.data

    @staticmethod
    def backward(ctx, grad_output):
        """
        Differentiates the optimal x returned by the forward pass with respect
        to the ratings matrix that was given as input.
        """
        import numpy as np
        from torch.autograd import Variable
        x = ctx.x
        params = ctx.params
        xgrad = ctx.xgrad
        dxdr = ContinuousOptimizer.get_dxdr(x.detach().numpy(), -xgrad.detach().numpy(), params.detach(
        ).numpy(), ctx.get_dgradf_dparams, ctx.get_hessian, ctx.max_x)
        dxdr_t = torch.from_numpy(np.transpose(dxdr))
        out = torch.mm(dxdr_t.float(), grad_output.view(len(x), 1))
        return out.view_as(params), None, None, None, None

    @staticmethod
    def get_dxdr(x, grad, params, get_dgradf_dparams, get_hessian, max_x):
        '''
        Returns the derivative of the optimal solution in the region around x in 
        terms of the rating matrix r. 

        x: an optimal solution

        grad: df/dx at x

        params: the current parameter settings
        '''
        import numpy as np
        import scipy as sp
        import scipy.sparse
        import scipy.linalg
        n = len(x)
        # first get the optimal dual variables via the KKT conditions
        # dual variable for constraint sum(x) <= k
        if np.logical_and(x > 0, x < max_x).any():
            lambda_sum = np.mean(grad[np.logical_and(x > 0, x < max_x)])
        else:
            lambda_sum = 0
        # dual variable for constraint x <= max_x
        lambda_upper = []
        # dual variable for constraint x >= 0
        lambda_lower = []
        for i in range(n):
            if np.abs(x[i] - max_x) < 0.000001:
                lambda_upper.append(grad[i] - lambda_sum)
            else:
                lambda_upper.append(0)
            if x[i] > 0:
                lambda_lower.append(0)
            else:
                lambda_lower.append(grad[i] - lambda_sum)
        # number of constraints
        m = 2*n + 1
        # collect value of dual variables
        lam = np.zeros((m))
        lam[0] = lambda_sum
        lam[1:(n+1)] = lambda_upper
        lam[n+1:] = lambda_lower
        diag_lambda = np.matrix(np.diag(lam))
        # collect value of constraints
        g = np.zeros((m))
        # TODO: replace the second x.sum() with k so that this is actually generally correct
        g[0] = x.sum() - x.sum()
        g[1:(n+1)] = x - max_x
        g[n+1:] = -x
        diag_g = np.matrix(np.diag(g))
        # gradient of constraints wrt x
        dgdx = np.zeros((m, n))
        # gradient of constraint sum(x) <= k
        dgdx[0, :] = 1
        # gradient of constraints x <= 1
        for i in range(1, n+1):
            dgdx[i, i-1] = 1
        # gradient of constraints x >= 0 <--> -x <= 0
        for i in range(n+1, m):
            dgdx[i, i-(n+1)] = -1
        dgdx = np.matrix(dgdx)
        # the Hessian matrix -- all zeros for now
        if get_hessian == None:
            H = np.matrix(np.zeros((n, n)))
        else:
            H = get_hessian(x, params)
        # coefficient matrix for the linear system
        A = np.bmat([[H, np.transpose(dgdx)], [diag_lambda*dgdx, diag_g]])
        # add 0.01*I to improve conditioning
        A = A + 0.01*np.eye(n+m)
        # RHS of the linear system, mostly partial derivative of grad f wrt params
        dgradf_dparams = get_dgradf_dparams(x, params, num_samples=1000)
        reshaped = np.zeros(
            (dgradf_dparams.shape[0], dgradf_dparams.shape[1]*dgradf_dparams.shape[2]))
        for i in range(n):
            reshaped[i] = dgradf_dparams[i].flatten()
        b = np.bmat([[reshaped], [np.zeros((m, reshaped.shape[1]))]])
        # solution to the system
        derivatives = sp.linalg.solve(A, b)
        if np.isnan(derivatives).any():
            print('report')
            print(np.isnan(A).any())
            print(np.isnan(b).any())
            print(np.isnan(dgdx).any())
            print(np.isnan(diag_lambda).any())
            print(np.isnan(diag_g).any())
            print(np.isnan(dgradf_dparams).any())
        # first n are derivatives of primal variables
        derivatives = derivatives[:n]
        return derivatives


In [3]:
# coverage.py
import torch
import numpy as np
from numba import jit


@jit
def gradient_coverage(x, P, w):
    n = len(w)
    m = len(x)
    grad = np.zeros(m, dtype=np.float32)
    for i in range(n):
        p_fail = 1 - x*P[:, i]
        p_all_fail = np.prod(p_fail)
        for j in range(m):
            grad[j] += w[i] * P[j, i] * p_all_fail/p_fail[j]
    return grad


@jit
def hessian_coverage(x, P, w):
    n = len(w)
    m = len(x)
    hessian = np.zeros((m, m), dtype=np.float32)
    for i in range(n):
        p_fail = 1 - x*P[:, i]
        p_all_fail = np.prod(p_fail)
        for j in range(m):
            for k in range(m):
                hessian[j, k] = -w[i] * P[j, i] * \
                    p_all_fail/(p_fail[j] * p_fail[k])
    return hessian


@jit
def objective_coverage(x, P, w):
    n = len(w)
    total = 0
    for i in range(n):
        p_fail = 1 - x*P[:, i]
        p_all_fail = np.prod(p_fail)
        total += w[i] * (1 - p_all_fail)
    return total


class CoverageInstanceMultilinear(torch.autograd.Function):
    """
    Represents a coverage instance with given coverage probabilities
    P and weights w. Forward pass computes the objective value (if evaluate_forward
    is true). Backward computes the gradients wrt decision variables x.
    """
    @staticmethod
    def forward(ctx, x, P, w, evaluate_forward):
        ctx.evaluate_forward = evaluate_forward
        if type(P) != np.ndarray:
            P = P.detach().numpy()
        if type(w) != np.ndarray:
            w = w.detach().numpy()
        ctx.P = P
        ctx.w = w

        ctx.x = x.detach().numpy()
        if ctx.evaluate_forward:
            out = objective_coverage(ctx.x, ctx.P, ctx.w)
        else:
            out = -1
        return torch.tensor(out).float()

    @staticmethod
    def backward(ctx, grad_in):
        grad = gradient_coverage(ctx.x, ctx.P, ctx.w)
        return torch.from_numpy(grad).float()*grad_in.float(), None, None, None


def optimize_coverage_multilinear(P, w, verbose=True, k=10, c=1., minibatch_size=None):
    '''
    Run some variant of SGD for the coverage problem with given 
    coverage probabilities P and weights w

    '''
    import torch
    # from utils import project_uniform_matroid_boundary as project

    # objective which will provide gradient evaluations
    # coverage = CoverageInstanceMultilinear.apply(P, w, verbose) # move to call below
    # decision variables
    x = torch.zeros(P.shape[0], requires_grad=True)
    # set up the optimizer
    learning_rate = 0.1
    optimizer = torch.optim.SGD(
        [x], momentum=0.9, lr=learning_rate, nesterov=True)
    # take projected stochastic gradient steps
    for t in range(10):
        loss = -CoverageInstanceMultilinear.apply(x, P, w, verbose)
        if verbose:
            print(t, -loss.item())
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        x.data = torch.from_numpy(project_uniform_matroid_boundary(x.data.numpy(), k, 1/c)).float()
    return x


@jit
def dgrad_coverage(x, P, num_samples, w):
    n = len(w)
    m = len(x)
    dgrad = np.zeros((m, m, n), dtype=np.float32)
    for i in range(n):
        p_fail = 1 - x*P[:, i]
        p_all_fail = np.prod(p_fail)
        for j in range(m):
            for k in range(m):
                if j == k:
                    dgrad[j, k, i] = w[i] * p_all_fail/p_fail[j]
                else:
                    dgrad[j, k, i] = -w[i] * x[k] * P[j, i] * \
                        p_all_fail/(p_fail[j] * p_fail[k])
    return dgrad


@jit
def dgrad_coverage_stochastic(x, P, num_samples, w, num_real_samples):
    n = len(w)
    m = len(x)
    rand_rows = np.random.choice(list(range(m)), num_real_samples)
    rand_cols = np.random.choice(list(range(n)), num_real_samples)

    dgrad = np.zeros((m, m, n), dtype=np.float32)

    p_fail = np.zeros((n, m), dtype=np.float32)
    p_all_fail = np.zeros((n), dtype=np.float32)
    for i in range(n):
        p_fail[i] = 1 - x*P[:, i]
        p_all_fail[i] = np.prod(p_fail[i])

    for sample in range(num_real_samples):
        k = rand_rows[sample]
        i = rand_cols[sample]
        for j in range(m):
            if j == k:
                dgrad[j, k, i] = w[i] * p_all_fail[i]/p_fail[i, j]
            else:
                dgrad[j, k, i] = -w[i] * x[k] * P[j, i] * \
                    p_all_fail[i]/(p_fail[i, j] * p_fail[i, k])
    return dgrad


In [4]:
## utils.py

def project_uniform_matroid_boundary(x, k, c=1):
    '''
    Exact projection algorithm of Karimi et al. This is the projection implementation
    that should be used now.
    
    Projects x onto the set {y: 0 <= y <= 1/c, ||y||_1 = k}
    '''
    import numpy as np
    k *= c
    n = len(x)
    x = x.copy()
    alpha_upper = x/c
    alpha_lower = (x*c - 1)/c**2
    S = []
    S.extend(alpha_lower)
    S.extend(alpha_upper)
    S.sort()
    S = np.unique(S)
    h = n
    alpha = min(S) - 1
    m = 0
    for i in range(len(S)):
        hprime = h + (S[i] - alpha)*m
        if hprime < k and k <= h:
            alphastar = (S[i] - alpha)*(h - k)/(h - hprime) + alpha
            result = np.zeros((n))
            for j in range(n):
                if alpha_lower[j] > alphastar:
                    result[j] = 1./c
                elif alpha_upper[j] >= alphastar:
                    result[j] = x[j] - alphastar*c
            return result
        m -= (alpha_lower == S[i]).sum()*(c**2)
        m += (alpha_upper == S[i]).sum()*(c**2)
        h = hprime
        alpha = S[i]
    raise Exception('projection did not terminate')

def project_cvx(x, k):
    '''
    Exact Euclidean projection onto the boundary of the k uniform matroid polytope.
    '''
    from cvxpy import Variable, Minimize, sum_squares, Problem
    import numpy as np
    n = len(x)
    p = Variable(n, 1)
    objective = Minimize(sum_squares(p - x))
    constraints = [sum(p) == k, p >= 0, p <= 1]
    prob = Problem(objective, constraints)
    prob.solve()
    return np.reshape(np.array(p.value), x.shape)


### Load & Preprocess Data



The probabilities of coverage below follows the rules:


*   1 can match with 4
*   1 and 4 never match with 2
*   3 never matches with 1, 2 or 4.

Thus, it becomes impossible to cover all 4 targets if k = 2. The highest objective value we can obtain is 3.


In [5]:
# load probability matrix 
P_list = [
  [0,0,0,1],
  [0,1,0,0],
  [1,0,0,1],
  [0,1,0,0],
  [0,0,0,0],
  [0,0,1,0],
  [0,0,0,0],
  [0,1,0,0],
  [0,0,0,1],
  [0,1,0,0],
  [0,0,1,0],
  [1,0,0,1],
  [0,0,1,0],
  [1,0,0,1],
  [1,0,0,1],
  [0,0,0,1],
  [0,0,0,1],
  [0,0,0,1],
  [0,0,0,1],
  [0,0,0,0],
  [0,0,0,0],
  [0,0,0,1],
  [0,0,0,1],
  [1,0,0,1],
  [0,0,0,0],
  [0,0,0,1]
]



In [7]:
# load features
circuit_km = [3.7, 
3, 
6.9, 
3.5, 
2, 
2.8, 
2.4, 
4.5, 
4.3, 
2.6, 
3.2, 
4.9, 
1.9, 
4.1, 
3.4, 
4.3, 
4, 
3.3, 
3.4, 
2.4, 
1.9, 
4.5, 
4, 
4.9, 
1.2, 
3.9, 
]
y = []
for i in circuit_km:
  y.append([i])

In [8]:
## recommendation_nn_decision.py
import numpy as np
import torch
# from coverage import optimize_coverage_multilinear, CoverageInstanceMultilinear, dgrad_coverage, hessian_coverage
import pickle
from functools import partial
# from submodular import ContinuousOptimizer
import torch.nn as nn
import random
# import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument('--layers', type=int, default=1)
# parser.add_argument('--activation', type=str, default='relu')
# parser.add_argument('--k', type=int, default=20)

# args = parser.parse_args()
num_layers = 4
activation = 'relu'
k = 2
use_hessian = False
num_iters = 400
instance_sizes = [0]
learning_rate = 1e-4

Ps = {}
data = {}
f_true = {}

for num_items in instance_sizes:
    Ps_size = np.array(P_list)
    data_size = np.array(y)

    num_targets = Ps_size.shape[1] #500 --> 4
    num_features = data_size.shape[1] #2113 --> 1
    Ps[num_items] = [torch.from_numpy(Ps_size).long()]
    data[num_items] = [torch.from_numpy(data_size).float()]
    w = np.ones(num_targets, dtype=np.float32)
    f_true[num_items] = [(P, w) for P in Ps[num_items]]
  
num_repetitions = 0

train = {}
test = {}
for size in instance_sizes:
  train[size], test[size] = np.array(P_list), np.array(y)

### Train NN

In [9]:
vals = np.zeros((num_repetitions+1, len(instance_sizes), len(instance_sizes)))

for idx in range(num_repetitions, num_repetitions + 1):

    intermediate_size = 26

    def make_fc():
        if num_layers > 1:
            if activation == 'relu':
                activation_fn = nn.ReLU
            elif activation == 'sigmoid':
                activation_fn = nn.Sigmoid
            else:
                raise Exception(
                    'Invalid activation function: ' + str(activation))
            net_layers = [
                nn.Linear(num_features, intermediate_size), activation_fn()]
            for hidden in range(num_layers-2):
                net_layers.append(
                    nn.Linear(intermediate_size, intermediate_size))
                net_layers.append(activation_fn())
            net_layers.append(nn.Linear(intermediate_size, num_targets))
            net_layers.append(nn.Sigmoid())
            return nn.Sequential(*net_layers)
        else:
            return nn.Sequential(nn.Linear(num_features, num_targets), nn.Sigmoid())

    # optimizer that will be used for training (and testing)
    optfunc = partial(optimize_coverage_multilinear, w=w, k=k, c=0.95)
    dgrad = partial(dgrad_coverage, w=w)
    if use_hessian:
        hessian = partial(hessian_coverage, w=w)
    else:
        hessian = None

    # runs the given net on instances of a given size
    def eval_opt(net, instances, size):
        net.eval()
        val = 0.
        for i in range(len(instances)):
            pred = net(data[size][i])
            x = ContinuousOptimizer.apply(pred, optfunc, dgrad, None, 0.95)
            pp, ww = f_true[size][i]
            val += objective_coverage(x.detach().numpy(), pp.detach().numpy(), ww)
        net.train()
        return val/len(instances), pred, x

    # train a network for each size, and test on each sizes
    for train_idx, train_size in enumerate(instance_sizes):
        net = make_fc()
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
        # training
        for t in range(num_iters):
            print(f"Iteration {t}")
            #i = random.randint(0, 80)
            i=0
            pred = net(data[train_size][i])
            x = ContinuousOptimizer.apply(pred, optfunc, dgrad, None, 0.95)
            pp, ww = f_true[train_size][i]
            loss = -CoverageInstanceMultilinear.apply(x, pp, ww, False)
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
        # save learned network state
        savepath = '/tmp/net_diffopt_smalllr_{0}_{1}_{2}_{3}.pt'.format(
            train_size, k, num_layers, idx)
        torch.save(net.state_dict(), savepath)
        # test on different sizes
        for test_idx, test_size in enumerate(instance_sizes):
            vals[idx, train_idx, test_idx], return_prediction, return_x = eval_opt(net, test, test_size)
            print(vals[idx, train_idx, test_idx])
        # save out values
        print(idx, train_size, vals[idx, train_idx])
        with open('results_recommendation_' + str(num_layers) + '.pickle', 'wb') as f:
            pickle.dump(vals, f)


Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Iteration 21
Iteration 22
Iteration 23
Iteration 24
Iteration 25
Iteration 26
Iteration 27
Iteration 28
Iteration 29
Iteration 30
Iteration 31
Iteration 32
Iteration 33
Iteration 34
Iteration 35
Iteration 36
Iteration 37
Iteration 38
Iteration 39
Iteration 40
Iteration 41
Iteration 42
Iteration 43
Iteration 44
Iteration 45
Iteration 46
Iteration 47
Iteration 48
Iteration 49
Iteration 50
Iteration 51
Iteration 52
Iteration 53
Iteration 54
Iteration 55
Iteration 56
Iteration 57
Iteration 58
Iteration 59
Iteration 60
Iteration 61
Iteration 62
Iteration 63
Iteration 64
Iteration 65
Iteration 66
Iteration 67
Iteration 68
Iteration 69
Iteration 70
Iteration 71
Iteration 72
Iteration 73
Iteration 74
Iteration 75
Iteration 76
Iteration

In [10]:
data[train_size][0]

tensor([[3.7000],
        [3.0000],
        [6.9000],
        [3.5000],
        [2.0000],
        [2.8000],
        [2.4000],
        [4.5000],
        [4.3000],
        [2.6000],
        [3.2000],
        [4.9000],
        [1.9000],
        [4.1000],
        [3.4000],
        [4.3000],
        [4.0000],
        [3.3000],
        [3.4000],
        [2.4000],
        [1.9000],
        [4.5000],
        [4.0000],
        [4.9000],
        [1.2000],
        [3.9000]])

In [11]:
return_prediction

tensor([[0.6461, 0.5866, 0.5300, 0.5249],
        [0.6223, 0.5761, 0.5225, 0.5156],
        [0.7565, 0.6400, 0.5505, 0.5655],
        [0.6393, 0.5835, 0.5279, 0.5222],
        [0.5875, 0.5611, 0.5119, 0.5025],
        [0.6154, 0.5731, 0.5204, 0.5130],
        [0.6015, 0.5671, 0.5161, 0.5078],
        [0.6737, 0.5997, 0.5384, 0.5360],
        [0.6667, 0.5963, 0.5364, 0.5332],
        [0.6085, 0.5701, 0.5183, 0.5104],
        [0.6292, 0.5790, 0.5247, 0.5183],
        [0.6885, 0.6065, 0.5407, 0.5410],
        [0.5839, 0.5596, 0.5108, 0.5012],
        [0.6596, 0.5929, 0.5343, 0.5304],
        [0.6360, 0.5820, 0.5268, 0.5209],
        [0.6667, 0.5963, 0.5364, 0.5332],
        [0.6561, 0.5912, 0.5332, 0.5290],
        [0.6326, 0.5805, 0.5257, 0.5196],
        [0.6360, 0.5820, 0.5268, 0.5209],
        [0.6015, 0.5671, 0.5161, 0.5078],
        [0.5839, 0.5596, 0.5108, 0.5012],
        [0.6737, 0.5997, 0.5384, 0.5360],
        [0.6561, 0.5912, 0.5332, 0.5290],
        [0.6885, 0.6065, 0.5407, 0

In [12]:
return_x

tensor([0.0000, 0.0000, 0.9500, 0.0000, 0.0000, 0.0000, 0.0000, 0.1484, 0.0905,
        0.0000, 0.0000, 0.2621, 0.0000, 0.0342, 0.0000, 0.0905, 0.0070, 0.0000,
        0.0000, 0.0000, 0.0000, 0.1484, 0.0070, 0.2621, 0.0000, 0.0000],
       grad_fn=<ContinuousOptimizerBackward>)

In [13]:
return_x.round()

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<RoundBackward0>)

In [14]:
pred = net(data[train_size][0])
pred

tensor([[0.6461, 0.5866, 0.5300, 0.5249],
        [0.6223, 0.5761, 0.5225, 0.5156],
        [0.7565, 0.6400, 0.5505, 0.5655],
        [0.6393, 0.5835, 0.5279, 0.5222],
        [0.5875, 0.5611, 0.5119, 0.5025],
        [0.6154, 0.5731, 0.5204, 0.5130],
        [0.6015, 0.5671, 0.5161, 0.5078],
        [0.6737, 0.5997, 0.5384, 0.5360],
        [0.6667, 0.5963, 0.5364, 0.5332],
        [0.6085, 0.5701, 0.5183, 0.5104],
        [0.6292, 0.5790, 0.5247, 0.5183],
        [0.6885, 0.6065, 0.5407, 0.5410],
        [0.5839, 0.5596, 0.5108, 0.5012],
        [0.6596, 0.5929, 0.5343, 0.5304],
        [0.6360, 0.5820, 0.5268, 0.5209],
        [0.6667, 0.5963, 0.5364, 0.5332],
        [0.6561, 0.5912, 0.5332, 0.5290],
        [0.6326, 0.5805, 0.5257, 0.5196],
        [0.6360, 0.5820, 0.5268, 0.5209],
        [0.6015, 0.5671, 0.5161, 0.5078],
        [0.5839, 0.5596, 0.5108, 0.5012],
        [0.6737, 0.5997, 0.5384, 0.5360],
        [0.6561, 0.5912, 0.5332, 0.5290],
        [0.6885, 0.6065, 0.5407, 0

In [15]:
f_true[0][0][0]-pred

tensor([[-0.6461, -0.5866, -0.5300,  0.4751],
        [-0.6223,  0.4239, -0.5225, -0.5156],
        [ 0.2435, -0.6400, -0.5505,  0.4345],
        [-0.6393,  0.4165, -0.5279, -0.5222],
        [-0.5875, -0.5611, -0.5119, -0.5025],
        [-0.6154, -0.5731,  0.4796, -0.5130],
        [-0.6015, -0.5671, -0.5161, -0.5078],
        [-0.6737,  0.4003, -0.5384, -0.5360],
        [-0.6667, -0.5963, -0.5364,  0.4668],
        [-0.6085,  0.4299, -0.5183, -0.5104],
        [-0.6292, -0.5790,  0.4753, -0.5183],
        [ 0.3115, -0.6065, -0.5407,  0.4590],
        [-0.5839, -0.5596,  0.4892, -0.5012],
        [ 0.3404, -0.5929, -0.5343,  0.4696],
        [ 0.3640, -0.5820, -0.5268,  0.4791],
        [-0.6667, -0.5963, -0.5364,  0.4668],
        [-0.6561, -0.5912, -0.5332,  0.4710],
        [-0.6326, -0.5805, -0.5257,  0.4804],
        [-0.6360, -0.5820, -0.5268,  0.4791],
        [-0.6015, -0.5671, -0.5161, -0.5078],
        [-0.5839, -0.5596, -0.5108, -0.5012],
        [-0.6737, -0.5997, -0.5384

In [16]:
pred.round()

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 0., 0.],
        [1., 1., 1., 1.]], grad_fn=<RoundBackward0>)

In [17]:
pp, ww = f_true[size][i]

In [18]:
x = ContinuousOptimizer.apply(pred, optfunc, dgrad, None, 0.95)

In [19]:
x

tensor([0.0000, 0.0000, 0.9500, 0.0000, 0.0000, 0.0000, 0.0000, 0.1484, 0.0905,
        0.0000, 0.0000, 0.2621, 0.0000, 0.0342, 0.0000, 0.0905, 0.0070, 0.0000,
        0.0000, 0.0000, 0.0000, 0.1484, 0.0070, 0.2621, 0.0000, 0.0000],
       grad_fn=<ContinuousOptimizerBackward>)

In [20]:
n = len(ww)
total = 0
for i in range(n):
  p_fail = 1 - return_x.detach().numpy()*return_prediction.detach().numpy()[:, i]
  p_all_fail = np.prod(p_fail)
  total += ww[i] * (1 - p_all_fail)

In [21]:
total

3.151599705219269

In [22]:
objective_coverage(x.detach().numpy(), pp.detach().numpy(), ww)

2.1038407869637012

In [23]:
x.topk(2)

torch.return_types.topk(
values=tensor([0.9500, 0.2621], grad_fn=<TopkBackward0>),
indices=tensor([ 2, 23]))